<a href="https://colab.research.google.com/github/sofman98/pfe/blob/master/notre_approche.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**generating el datasetto**

In [0]:
import pandas as pn
import numpy as np

def dataset_split(dataset,train_neg,test_neg):

    ratings = dataset.pivot(index = 'user_id', columns = 'item_id', values = 'rating')
    ratings.fillna(0, inplace=True)
    ratings=np.matrix(ratings)

    num_users, num_items = np.shape(ratings)
    
    #POSITIVES
        #test    
    testpos=[]
    for u in range(num_users):
        i = np.random.randint(num_items)
        while(ratings[u,i]==0):
            i = np.random.randint(num_items)
            testpos.append([[u,i,1]])
        if(u==0):
            indexes = dataset[(dataset.user_id==u) & (dataset.item_id==i)].index
        else:
            indexes = indexes.append(dataset[(dataset.user_id==u) & (dataset.item_id==i)].index)
    testpos = np.vstack(testpos)
    test=pn.DataFrame(data=testpos,columns=['user_id','item_id','rating'])

        #train
    dataset.drop(indexes , inplace=True)
    train = dataset
    
    #NEGATIVES
        #test
    testneg = []

    for u in range(num_users):
        for num_neg in range(test_neg):
            
            i = np.random.randint(num_items)
            while(ratings[u,i]>0):
                i = np.random.randint(num_items)
            ratings[u,i]=1            
            testneg.append([[u,i,0]])
    testneg = pn.DataFrame(data=np.vstack(testneg),columns=['user_id','item_id','rating'])

    test = test.append(testneg)
    test = test.sample(frac=1).reset_index(drop=True)

        #train
    trainneg=[]

    for x in range(train.shape[0]):
        for num_neg in range(train_neg):
            
            i = np.random.randint(num_items)
            u = np.random.randint(num_users)
            while(ratings[u,i]>0):
                i = np.random.randint(num_items)
            ratings[u,i]=1            
            trainneg.append([[u,i,0]])
    
    trainneg = pn.DataFrame(data=np.vstack(trainneg),columns=['user_id','item_id','rating'])

    train = train.append(trainneg)
    train = train.sample(frac=1).reset_index(drop=True)
    return train,test


def genres_items(items):
    
    result = []
    for index, row in items.iterrows():
            genre = row['genre']
            vec = np.zeros(19).astype(int)
            vec[0]=row['item_id']
            if 'Action' in genre:
                vec[1]=1
            if 'Adventure' in genre:
                vec[2]=1
                
            if 'Animation' in genre:
                vec[3]=1
                
            if 'Children\'s' in genre:
                vec[4]=1
                
            if 'Comedy' in genre:
                vec[5]=1
                
            if 'Crime' in genre:
                vec[6]=1
                
            if 'Documentary' in genre:
                vec[7]=1
                
            if 'Drama' in genre:
                vec[8]=1
                
            if 'Fantasy' in genre:
                vec[9]=1
                
            if 'Film-Noir' in genre:
                vec[10]=1
                
            if 'Horror' in genre:
                vec[11]=1
                
            if 'Musical' in genre:
                vec[12]=1
                
            if 'Mystery' in genre:
                vec[13]=1
                
            if 'Romance' in genre:
                vec[14]=1
                
            if 'Sci-Fi' in genre:
                vec[15]=1
                
            if 'Thriller' in genre:
                vec[16]=1
                
            if 'War' in genre:
                vec[17]=1
                
            if 'Western' in genre:
                vec[18]=1

            result.append(vec)
    result=np.vstack(result)

    new_items = pn.DataFrame(data=result, columns = ['item_id',
     'Action' ,
     'Adventure',
	 'Animation',
	 'Children\'s',
	 'Comedy',
	 'Crime',
	 'Documentary',
	 'Drama',
	 'Fantasy',
	 'Film-Noir',
	 'Horror',
	 'Musical',
	 'Mystery',
	 'Romance',
	 'Sci-Fi',
	 'Thriller',
	 'War',
	 'Western'],)

    #items = pn.merge(items,new_items, left_on='item_id', right_on='item_id', how='left')
    #items.drop('genre', axis=1, inplace=True)
    #items.drop('title', axis=1, inplace=True)

    return items

users = pn.read_csv('https://raw.githubusercontent.com/sofman98/pfe/master/users_encoded.csv', delimiter = '::',encoding ='cp1252',
    names = ['user_id','sexe', 'age','occupation'], engine = 'python')
    

items = pn.read_csv('https://raw.githubusercontent.com/sofman98/pfe/master/movies_encoded.csv', delimiter = '::',encoding ='cp1252',
    names = ['item_id','title','genre'], engine = 'python')

items=genres_items(items)

ratings = pn.read_csv('https://raw.githubusercontent.com/sofman98/pfe/master/ratings.csv', delimiter = '::',encoding ='cp1252',
    names = ['user_id', 'item_id', 'rating', 'timestamp'], engine = 'python')
ratings.drop('timestamp', axis=1, inplace=True)
ratings['rating']=1


ratings.user_id = ratings.user_id.astype('category').cat.codes.values
ratings.item_id = ratings.item_id.astype('category').cat.codes.values


train,test = dataset_split(ratings,4,99)


train = pn.merge(train,users, left_on='user_id', right_on='user_id', how='left')
train = pn.merge(train,items, left_on='item_id', right_on='item_id', how='left')

test = pn.merge(test,users, left_on='user_id', right_on='user_id', how='left')
test = pn.merge(test,items, left_on='item_id', right_on='item_id', how='left')

np.savetxt('train.csv',train, delimiter='::',fmt='%s')
np.savetxt('test.csv',test, delimiter='::',fmt='%s')

'''
items = ratings.loc[:,['item_id','title','genre']]
items = items.drop_duplicates()
items = items.sort_values(by=['item_id'])

users = ratings.loc[:,['user_id','sexe', 'age','occupation']]
users = users.drop_duplicates()
users = users.sort_values(by=['user_id'])


np.savetxt('C:/Users/Zeghoud/Desktop/Datasets/ml-1m/movies_encoded.csv',items, delimiter='::',fmt='%s')

np.savetxt('C:/Users/Zeghoud/Desktop/Datasets/ml-1m/users_encoded.csv',users, delimiter='::',fmt='%d')


'''

#**code-notre approche**

In [21]:
import pandas as pn
import numpy as np
import math
from keras.utils.vis_utils import model_to_dot
from keras.layers.core import Flatten
from keras.layers import Embedding, Dropout, Multiply, Dense, Input
from keras.models import Model
from keras.layers.merge import concatenate
from keras.callbacks import Callback
from keras import backend as K
from keras.models import load_model
from time import time
import heapq

#ML 1m + content based

def evaluate_model(model,test, k):
    
    #test
    test_userID = test['user_id']
    test_userDATA = test[['sexe', 'age','occupation']]

    test_itemID = test['item_id']
    test_itemDATA = test.iloc[:,-18:]

    test_y = test['rating']

    
    predictions = model.predict([test_userID, test_userDATA, test_itemID, test_itemDATA], verbose=2)
    predictions = pn.DataFrame(data=predictions,columns=['predicted'])
    predictions = pn.concat([test,predictions], axis=1)
    predictions = predictions.sort_values(by=['predicted'], ascending=False)
    
    users=test.user_id.unique()
    
    hrs, ndcgs = 0,0
    for u in users:
        p= predictions[predictions['user_id']==u].loc[:,'rating']
        hr, ndcg = evaluate_by_user(p, k)
        hrs = hrs + hr
        ndcgs = ndcgs + ndcg
        
    mean_hr = hrs/len(users)
    mean_ndcg = ndcgs/len(users)
        
    return mean_hr, mean_ndcg


def evaluate_by_user(test, k):
    hr, ndcg = 0, 0
    i=0
    for rating in test:
        if rating == 1 :
            hr = 1
            ndcg = math.log(2)/math.log(i+2)
            break 
        i=i+1
        if(i==k):
            break
    
    return hr, ndcg
  
'''
def scores_K(prediction_list, item):
    hr, ndcg = 0, 0
    for i in prediction_list:
        if i == item :
            hr = 1
            ndcg = math.log(2)/math.log(i+2)
            break
    
    return hr, ndcg
'''

def load_data():
    trainset = pn.read_csv('train.csv', delimiter = '::',
    names = ['user_id', 'item_id', 'rating','sexe', 'age','occupation', 'Action' ,
    'Adventure', 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir',
    'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western'], engine = 'python')
    
    testset = pn.read_csv('test.csv', delimiter = '::',
    names = ['user_id', 'item_id', 'rating','sexe', 'age','occupation', 'Action' ,
    'Adventure', 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir',
    'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western'], engine = 'python')
     
    num_users = testset.user_id.max()+1
    num_items = trainset.item_id.max()+1
    
    return trainset, testset, num_users, num_items


train, test, num_users, num_items = load_data()

#train
train_userID = train['user_id']
train_userDATA = train[['sexe', 'age','occupation']]

train_itemID = train['item_id']
train_itemDATA = train.iloc[:,-18:]

train_y = train['rating']



#create model 
def build_model():
    input_userID = Input(shape = [1], name = 'user_ID')
    input_userDATA = Input(shape = [3], name = 'user_data')
    input_itemID = Input(shape = [1], name = 'item_ID')
    input_itemDATA = Input(shape = [18], name = 'item_data')

    #GMF part
    user_latent_factors_GMF = 32
    item_latent_factors_GMF = 32

    user_emb_GMF = Embedding(num_users, user_latent_factors_GMF, name = 'user_emb_GMF')(input_userID)
    item_emb_GMF = Embedding(num_items, item_latent_factors_GMF, name = 'item_emb_GMF')(input_itemID)

    flat_u_GMF = Flatten()(user_emb_GMF)
    flat_i_GMF = Flatten()(item_emb_GMF)

    mul_layer = Multiply()([flat_u_GMF, flat_i_GMF])

    #MLP part
    user_latent_factors_MLP = 32
    item_latent_factors_MLP = 32

    user_emb_MLP = Embedding(num_users, user_latent_factors_MLP, name= 'user_emb_MLP')(input_userID)
    item_emb_MLP = Embedding(num_items, item_latent_factors_MLP, name= 'item_emb_MLP')(input_itemID)

    flat_u_MLP = Flatten()(user_emb_MLP)
    flat_i_MLP = Flatten()(item_emb_MLP)

    concat_MLP = concatenate([flat_u_MLP, flat_i_MLP, input_userDATA, input_itemDATA])

    layer1 = Dense(64, activation='relu')(concat_MLP)
    layer2 = Dense(32, activation='relu')(layer1)
    last_layer_MLP = Dense(16, activation='relu')(layer2)

    #NeuMF
    concat_NeuMF = concatenate([mul_layer,last_layer_MLP])

    out = Dense(1, activation='sigmoid', name='output')(concat_NeuMF)

    model = Model([input_userID, input_userDATA, input_itemID, input_itemDATA], out)
    
    return model


model = build_model()

model.summary()

model.compile(optimizer = 'sgd', loss = 'binary_crossentropy')

#evaluation of the model without pretrain
#best_hr, best_ndcg, best_iteration = 0, 0, -1
#verbose = 1 #show performance every 1 epoch
#for every epoch
for epoch in range(2):
    t1 = time()
    history = model.fit([train_userID, train_userDATA, train_itemID, train_itemDATA], train_y, batch_size=100 , epochs=1, verbose=0, shuffle=True)
    t2 = time()
    if epoch % verbose == 0 :
        hr, ndcg = evaluate_model(model, test, 10)
        print('Iteration %d [%.1f s]: loss = %.4f., HR = %.4f, NDCG = %.4f' % (epoch, t2-t1, history.history['loss'][0], hr, ndcg))
        if hr > best_hr :
            best_hr, best_ndcg, best_iteration = hr, ndcg, epoch
            model.save_weights('weights_best_iteration.hdf5', overwrite = True)
    
print("Best iteration %d, best HR = %.4f, best NDCG = %.4f" % (best_iteration, best_hr, best_ndcg))


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
user_ID (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
item_ID (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
user_emb_MLP (Embedding)        (None, 1, 32)        193280      user_ID[0][0]                    
__________________________________________________________________________________________________
item_emb_MLP (Embedding)        (None, 1, 32)        118592      item_ID[0][0]                    
__________________________________________________________________________________________________
flatten_39

KeyboardInterrupt: ignored